In [1]:
import simuOpt
simuOpt.setOptions(alleleType='short', quiet=True, numThreads=4)
import simuPOP as sim
import numpy as np
import pandas as pd
import random
import h5py
from saegus import analyze, operators, parameters, parse, breed
np.set_printoptions(suppress=True, precision=5)

## Data Storage

In [2]:
data_file = h5py.File("822_selection_data_example.hdf5")
allele_group = data_file['allele']
data_file['allele'].attrs['columns'] = list(map(np.string_, ['locus', 'alpha', 'omega', 'minor', 'major']))
trait_group = data_file['trait']

In [3]:
example_pop = sim.loadPopulation('example_pop.pop')
example_pop.addInfoFields(['ind_id', 'mother_id', 'father_id', 'g', 'p', 'sample', 'selection'])
example_pop.dvars().gen = 0
sim.tagID(example_pop)
sim.stat(example_pop, numOfSegSites=sim.ALL_AVAIL, 
         vars=['numOfSegSites','segSites', 'fixedSites'])
sim.stat(example_pop, alleleFreq=sim.ALL_AVAIL)
segregating_loci = example_pop.dvars().segSites

In [4]:
tf = parse.TusonFounders()

In [5]:
recom_map = tf.parse_recombination_rates('genetic_map.txt')

In [6]:
allele_states = analyze.gather_allele_data(example_pop)
allele_frequencies = analyze.gather_allele_frequencies(example_pop, allele_states)
gwas = analyze.GWAS(example_pop, np.array(segregating_loci, dtype=np.int_), allele_states[:, 3], 'example')

## Determine Sample

In [ ]:
sampled_inds = random.sample(example_pop.indInfo('ind_id'), 50)
for ind in sampled_inds:
    example_pop.indByID(ind).sample = 1

In [ ]:
sample_splitter = sim.InfoSplitter('sample', values=[1, 0], names=['sampled', 'not_sampled'])

In [ ]:
example_pop.setVirtualSplitter(sample_splitter)

In [ ]:
example_pop.numVirtualSubPop()

In [ ]:
example_pop.subPopName([0, 0])

In [ ]:
example_pop.subPopName([0, 1])

In [8]:
sampled_inds

[46.0,
 3.0,
 87.0,
 80.0,
 42.0,
 67.0,
 83.0,
 93.0,
 24.0,
 6.0,
 69.0,
 17.0,
 75.0,
 5.0,
 30.0,
 39.0,
 40.0,
 52.0,
 21.0,
 13.0,
 77.0,
 4.0,
 26.0,
 22.0,
 73.0,
 62.0,
 81.0,
 41.0,
 45.0,
 64.0,
 23.0,
 97.0,
 94.0,
 16.0,
 33.0,
 44.0,
 56.0,
 2.0,
 51.0,
 12.0,
 9.0,
 15.0,
 72.0,
 90.0,
 104.0,
 53.0,
 89.0,
 70.0,
 61.0,
 29.0]

In [ ]:
sim.stat(example_pop, alleleFreq=sim.ALL_AVAIL, subPops=[(0, 0)],
       vars=['alleleFreq_sp'])

In [ ]:
example_pop.indInfo('sample')

In [7]:
sampled_inds = random.sample(example_pop.indInfo('ind_id'), 50)

In [11]:
trait = parameters.Trait()

In [12]:
qtl = random.sample(segregating_loci, 30)

In [13]:
ae_table = trait.construct_allele_effects_table(example_pop, qtl, random.expovariate, 1)

In [14]:
ae_array = trait.construct_ae_array(ae_table, qtl)

In [15]:
operators.calculate_g(example_pop, ae_array)

In [16]:
operators.calculate_error_variance(example_pop, 0.7)

In [17]:
example_pop.dvars().epsilon

7.8867205584231002

In [18]:
trait_group.attrs['error_variance'] = example_pop.dvars().epsilon

In [19]:
operators.calculate_p(example_pop)

In [20]:
example_pop.sortIndividuals('p')

### Top 5% of Population

In [ ]:
import imp

In [ ]:
imp.reload(breed)

In [9]:
example_pop.setVirtualSplitter(sim.ProportionSplitter([0.95, 0.05], names=['not_selected', 'selected']))

In [21]:
example_pop.indInfo('p', [0, 1])

(65.14708343534227,
 65.82392583941191,
 66.495476630889,
 67.09846623319753,
 68.21133284868888)

In [22]:
for ind in example_pop.individuals([0, 1]):
    ind.setSex(1)
for ind in example_pop.individuals([0, 0]):
    ind.setSex(2)

In [23]:
example_pop.numVirtualSubPop()

2

In [24]:
example_pop.subPopSize(subPop=[0, 1])

6

In [25]:
example_pop.subPopName([0, 1])

'selected'

In [ ]:
dirsel = breed.DirectionalSelection(example_pop, [0, 1])

In [ ]:
example_pop.activateVirtualSubPop()

In [ ]:
example_pop.numVirtualSubPop()

In [ ]:
example_pop.

In [ ]:
example_pop.popSize()

In [27]:
female_chooser = sim.RandomParentChooser(sexChoice=sim.FEMALE_ONLY)
female_chooser.initialize(example_pop, 0)

In [ ]:
def rand_chooser(pop, subPop):
    male_chooser = sim.RandomParentChooser(sexChoice=sim.MALE_ONLY)
    male_chooser.initialize(pop, subPop)
    while True:
        m = male_chooser.chooseParents()[0]

In [ ]:
mc = sim.RandomPar

In [ ]:
example_pop.evolve(
    preOps=[
        operators.HDF5Trait('p', trait_group)
    ],
    matingScheme=
    sim.HomoMating(
        sim.PolyParentsChooser(
            polySex=sim.MALE, polyNum=10
            ),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=recom_map),
        ]), subPopSize=100
    ),
    gen=1
)

In [ ]:
example_pop.popSize()

In [ ]:
example_pop.indInfo('p')

In [ ]:
ped = sim.Pedigree(example_pop)

In [ ]:
fam = ped.identifyFamilies()

In [ ]:
pedigree = np.array([example_pop.indInfo('father_id'),
          example_pop.indInfo('mother_id'),
          example_pop.indInfo('ind_id')]).T

In [ ]:
print(pedigree)

In [ ]:
operators.calculate_g(example_pop, ae_array)

In [ ]:
operators.calculate_p(example_pop)

In [ ]:
example_pop.indInfo('p')

In [ ]:
example

In [ ]:
ped.locateRelatives(sim.COMMON_OFFSPRING)